In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
import requests
import json
import time
from tqdm import tqdm

# 서울특별시 구별 경계 그리드 좌표

In [194]:
data = pd.read_csv('data/서울특별시 구별 경계 좌표.csv', encoding='cp949')
data

,long,lat,order,hole,piece,id,group,sidoname
0,127.008643,37.580468,1,False,1.0,0.0,0.1,종로구
1,127.008713,37.580451,2,False,1.0,0.0,0.1,종로구
2,127.008766,37.580443,3,False,1.0,0.0,0.1,종로구
3,127.008908,37.580424,4,False,1.0,0.0,0.1,종로구
4,127.009138,37.580394,5,False,1.0,0.0,0.1,종로구
...,...,...,...,...,...,...,...,...
380387,128.601830,37.081987,3001,False,1.0,126.0,126.1,영월군
380388,128.601881,37.082031,3002,False,1.0,126.0,126.1,영월군
380389,128.601929,37.082096,3003,False,1.0,126.0,126.1,영월군
380390,128.601948,37.082122,3004,False,1.0,126.0,126.1,영월군


In [225]:
grid = pd.read_csv('data/KMeans/전체_그리드(2020, cp949).csv', encoding='cp949')
grid

,X좌표(경도),Y좌표(위도),백화점 개수,버스정류장 개수,병원 개수,영화관 개수,제과제빵 개수,지하철 개수,카페 개수,편의점 개수,음식점 개수,어린이집_유치원 개수,초중고등학교 개수,대학교 개수
0,126.7595 ~ 126.7654,37.41 ~ 37.4141,0,0,0,0,0,0,0,0,0.0,0,0,0
1,126.7595 ~ 126.7654,37.4141 ~ 37.4182,0,0,0,0,0,0,0,0,0.0,0,0,0
2,126.7595 ~ 126.7654,37.4182 ~ 37.4223,0,0,0,0,0,0,0,0,0.0,0,0,0
3,126.7595 ~ 126.7654,37.4223 ~ 37.426399999999994,0,0,0,0,0,0,0,0,0.0,0,0,0
4,126.7595 ~ 126.7654,37.426399999999994 ~ 37.430499999999995,0,0,0,0,0,0,0,0,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5395,127.17840000000001 ~ 127.18430000000001,37.696999999999996 ~ 37.7011,0,0,0,0,0,0,0,0,0.0,0,0,0
5396,127.17840000000001 ~ 127.18430000000001,37.7011 ~ 37.7052,0,0,0,0,0,0,0,0,0.0,0,0,0
5397,127.17840000000001 ~ 127.18430000000001,37.7052 ~ 37.7093,0,0,0,0,0,0,0,0,0.0,0,0,0
5398,127.17840000000001 ~ 127.18430000000001,37.7093 ~ 37.7134,0,0,0,0,0,0,0,0,0.0,0,0,0


In [227]:
xy_df = pd.DataFrame()
xy_df['번째'] = grid.index
xy_df['X1'] = grid['X좌표(경도)'].str.split(' ~ ').str[0]
xy_df['X2'] = grid['X좌표(경도)'].str.split(' ~ ').str[1]
xy_df['Y1'] = grid['Y좌표(위도)'].str.split(' ~ ').str[0]
xy_df['Y2'] = grid['Y좌표(위도)'].str.split(' ~ ').str[1]
xy_df = xy_df.astype('float')
xy_df['번째'] = xy_df['번째'].astype('int64')
xy_df

,번째,X1,X2,Y1,Y2
0,0,126.7595,126.7654,37.4100,37.4141
1,1,126.7595,126.7654,37.4141,37.4182
2,2,126.7595,126.7654,37.4182,37.4223
3,3,126.7595,126.7654,37.4223,37.4264
4,4,126.7595,126.7654,37.4264,37.4305
...,...,...,...,...,...
5395,5395,127.1784,127.1843,37.6970,37.7011
5396,5396,127.1784,127.1843,37.7011,37.7052
5397,5397,127.1784,127.1843,37.7052,37.7093
5398,5398,127.1784,127.1843,37.7093,37.7134


In [234]:
# 구 이름 리스트
name = pd.read_csv('data/CCTV_자치구별(2020년, cp949).csv', encoding='cp949')
name = [''.join(x.split())for x in name['기관명']]
name

['강남구',
 '강동구',
 '강북구',
 '강서구',
 '관악구',
 '광진구',
 '구로구',
 '금천구',
 '노원구',
 '도봉구',
 '동대문구',
 '동작구',
 '마포구',
 '서대문구',
 '서초구',
 '성동구',
 '성북구',
 '송파구',
 '양천구',
 '영등포구',
 '용산구',
 '은평구',
 '종로구',
 '중구',
 '중랑구']

In [249]:
def xy_data_cnt(name) :
    df = data[data['sidoname'] == name]

    df = df.drop(['order', 'hole', 'piece', 'id', 'group'], axis=1)
    df.columns = ['X좌표', 'Y좌표', '자치구']

    df = df.drop_duplicates(['X좌표', 'Y좌표'], keep='first')

    cnt_list = []
    for i in df.index :
        X_coor = df['X좌표'][i]
        Y_coor = df['Y좌표'][i]

        result1 = xy_df[(xy_df["X1"] <= X_coor) & (xy_df["X2"] > X_coor)]
        result2 = result1[(result1["Y1"] <= Y_coor) & (result1["Y2"] > Y_coor)]

        if len(result2) == 0:
            cnt_list.append(0)
            continue

        cnt_list.append(result2["번째"].values[0])

    df['번째'] = cnt_list

    path = 'data/경계 좌표/구별/' + name + ' 경계별 그리드 위치.csv'
    df.to_csv(path, encoding='cp949', index=False)

In [250]:
for n in name :
    xy_data_cnt(n)

In [343]:
def count_grid(name) :
    path = 'data/경계 좌표/' + name + ' 경계별 그리드 위치.csv'
    
    data = pd.read_csv(path, encoding='cp949')
        
    data_cnt = data['번째'].unique()
    
    grid_cnt = pd.DataFrame()
    grid_cnt['번째'] = data_cnt
    
    x1_list = []
    x2_list = []
    y1_list = []
    y2_list = []

    for i in grid_cnt['번째'].values :
        x1_list.append(xy_df[xy_df.index == i]['X1'].values[0])
        x2_list.append(xy_df[xy_df.index == i]['X2'].values[0])
        y1_list.append(xy_df[xy_df.index == i]['Y1'].values[0])
        y2_list.append(xy_df[xy_df.index == i]['Y2'].values[0])

    grid_cnt['X1'] = x1_list
    grid_cnt['X2'] = x2_list
    grid_cnt['Y1'] = y1_list
    grid_cnt['Y2'] = y2_list
    
    data_cnt = grid_cnt['번째'].unique()

    grid_cnt = pd.DataFrame()
    grid_cnt['번째'] = data_cnt

    x1_list = []
    x2_list = []
    y1_list = []
    y2_list = []

    for i in grid_cnt['번째'].values :
        x1_list.append(xy_df[xy_df.index == i]['X1'].values[0])
        x2_list.append(xy_df[xy_df.index == i]['X2'].values[0])
        y1_list.append(xy_df[xy_df.index == i]['Y1'].values[0])
        y2_list.append(xy_df[xy_df.index == i]['Y2'].values[0])

    grid_cnt['X1'] = x1_list
    grid_cnt['X2'] = x2_list
    grid_cnt['Y1'] = y1_list
    grid_cnt['Y2'] = y2_list

    cnt = 0
    for i in range(len(grid_cnt)) :
        cnt += grid_cnt[(grid_cnt['X1']==grid_cnt['X1'][i])]['번째'].max() - grid_cnt[(grid_cnt['X1']==grid_cnt['X1'][i])]['번째'].min() + 1

    return cnt

In [344]:
count_list = []
for n in name :
    count_list.append(count_grid(n))
    
count_list

[796,
 534,
 738,
 1079,
 646,
 374,
 528,
 407,
 1198,
 797,
 433,
 391,
 434,
 490,
 1308,
 324,
 712,
 813,
 585,
 715,
 479,
 1012,
 781,
 213,
 502]

In [351]:
cnt_df = pd.DataFrame()
cnt_df['구'] = name
cnt_df['그리드_개수'] = count_list
cnt_df

,구,그리드_개수
0,강남구,796
1,강동구,534
2,강북구,738
3,강서구,1079
4,관악구,646
5,광진구,374
6,구로구,528
7,금천구,407
8,노원구,1198
9,도봉구,797


In [352]:
cnt_df.to_csv('data/경계 좌표/FIN/구별_그리드_개수.csv', index=False, encoding='cp949')

# 서울시 구 그리드별 점수 추가

In [ ]:
def grid_score(name) :
    path = 'data/경계 좌표/구별/' + name + ' 경계별 그리드 위치.csv'
    file = pd.read_csv(path, encoding='cp949')
    
    score_1 = pd.read_csv('data/KMeans/1인 가구 가중치(0.5단위).csv', encoding='cp949')
    score_2 = pd.read_csv('data/KMeans/2인 가구 가중치(0.5단위).csv', encoding='cp949')
    score_4 = pd.read_csv('data/KMeans/4인 가구 가중치(0.5단위).csv', encoding='cp949')

    f_list = file['번째']
    score_1person = []
    score_2person = []
    score_4person = []
    
    for f in f_list :
        score_1person.append(score_1[score_1.index == f]['합계'].values[0])
        score_2person.append(score_2[score_2.index == f]['합계'].values[0])
        score_4person.append(score_4[score_4.index == f]['합계'].values[0])
        
    file['1인 가구 점수'] = score_1person
    file['2인 가구 점수'] = score_2person
    file['4인 가구 점수'] = score_4person

    fin_path = 'data/경계 좌표/구별/그리드 점수/' + name + ' 그리드별 점수.csv'
    file.to_csv(fin_path, encoding='cp949', index=False)

In [ ]:
# 구 이름 리스트
name = pd.read_csv('data/CCTV_자치구별(2020년, cp949).csv', encoding='cp949')
name = [''.join(x.split())for x in name['기관명']]
name

In [ ]:
for n in name :
    grid_score(n)

# 서울시 자치구 가구별 점수

In [ ]:
# 구 이름 리스트
name = pd.read_csv('data/CCTV_자치구별(2020년, cp949).csv', encoding='cp949')
name = [''.join(x.split())for x in name['기관명']]
name

In [ ]:
for i in range(len(name)) :
    gu_name = name[i]
    path = 'data/경계 좌표/구별/그리드 점수/' + gu_name + ' 그리드별 점수.csv'
    file = pd.read_csv(path, encoding='cp949')

    data = file[['1인 가구 점수', '2인 가구 점수', '4인 가구 점수']]
    
    score_list = data.sum(axis=0)

    if i == 0 :
        gu_score = pd.DataFrame()
        gu_score['자치구'] = file['자치구'].unique()
        gu_score['1인 가구 점수'] = score_list[0]/len(file)
        gu_score['2인 가구 점수'] = score_list[1]/len(file)
        gu_score['4인 가구 점수'] = score_list[2]/len(file)
        
    elif i == 1 :
        gu_score2 = pd.DataFrame()
        gu_score2['자치구'] = file['자치구'].unique()
        gu_score2['1인 가구 점수'] = score_list[0]/len(file)
        gu_score2['2인 가구 점수'] = score_list[1]/len(file)
        gu_score2['4인 가구 점수'] = score_list[2]/len(file)
        
        gu_score_df = pd.concat([gu_score, gu_score2], axis=0).reset_index(drop=True)
    
    else :
        gu_score2 = pd.DataFrame()
        gu_score2['자치구'] = file['자치구'].unique()
        gu_score2['1인 가구 점수'] = score_list[0]/len(file)
        gu_score2['2인 가구 점수'] = score_list[1]/len(file)
        gu_score2['4인 가구 점수'] = score_list[2]/len(file)
        
        gu_score_df = pd.concat([gu_score_df, gu_score2], axis=0).reset_index(drop=True)
        
gu_score_df

In [ ]:
gu_score_df.to_csv('data/경계 좌표/구별/서울시 자치구 가구별 점수.csv', encoding='cp949', index=False)

# 서울특별시 동별 그리드 경계 좌표

In [428]:
data = pd.read_csv('data/서울특별시 동별 경계 좌표.csv', encoding='cp949')
data

,long,lat,order,hole,piece,id,group,sidoname
0,126.975560,37.589681,1,False,1,0,0.1,청운동
1,126.975491,37.589552,2,False,1,0,0.1,청운동
2,126.975456,37.589483,3,False,1,0,0.1,청운동
3,126.975444,37.589431,4,False,1,0,0.1,청운동
4,126.975424,37.589355,5,False,1,0,0.1,청운동
...,...,...,...,...,...,...,...,...
3549277,126.776775,33.307159,1994,False,1,5046,5046.1,표선면
3549278,126.776762,33.307160,1995,False,1,5046,5046.1,표선면
3549279,126.776530,33.307115,1996,False,1,5046,5046.1,표선면
3549280,126.776206,33.307096,1997,False,1,5046,5046.1,표선면


In [435]:
law_dong = pd.read_excel('data/행정구역코드(법정동코드).xls')
law_dong

,법정동코드,법정동명,폐지여부
0,1100000000,서울특별시,존재
1,1111000000,서울특별시 종로구,존재
2,1111010100,서울특별시 종로구 청운동,존재
3,1111010200,서울특별시 종로구 신교동,존재
4,1111010300,서울특별시 종로구 궁정동,존재
...,...,...,...
45944,5013032022,제주특별자치도 서귀포시 표선면 하천리,존재
45945,5013032023,제주특별자치도 서귀포시 표선면 성읍리,존재
45946,5013032024,제주특별자치도 서귀포시 표선면 가시리,존재
45947,5013032025,제주특별자치도 서귀포시 표선면 세화리,존재


In [436]:
law_dong['시'] = law_dong['법정동명'].str.split(' ').str[0]
law_dong['구'] = law_dong['법정동명'].str.split(' ').str[1]
law_dong['동'] = law_dong['법정동명'].str.split(' ').str[2]
law_dong

,법정동코드,법정동명,폐지여부,시,구,동
0,1100000000,서울특별시,존재,서울특별시,NaN,NaN
1,1111000000,서울특별시 종로구,존재,서울특별시,종로구,NaN
2,1111010100,서울특별시 종로구 청운동,존재,서울특별시,종로구,청운동
3,1111010200,서울특별시 종로구 신교동,존재,서울특별시,종로구,신교동
4,1111010300,서울특별시 종로구 궁정동,존재,서울특별시,종로구,궁정동
...,...,...,...,...,...,...
45944,5013032022,제주특별자치도 서귀포시 표선면 하천리,존재,제주특별자치도,서귀포시,표선면
45945,5013032023,제주특별자치도 서귀포시 표선면 성읍리,존재,제주특별자치도,서귀포시,표선면
45946,5013032024,제주특별자치도 서귀포시 표선면 가시리,존재,제주특별자치도,서귀포시,표선면
45947,5013032025,제주특별자치도 서귀포시 표선면 세화리,존재,제주특별자치도,서귀포시,표선면


In [443]:
result = law_dong[law_dong["법정동코드"].astype("str").str[:2] == "11"]
seoul_dong = result[result["동"].notnull()][result["폐지여부"] == "존재"]
seoul_dong

<ipython-input-443-724f99014044>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  seoul_dong = result[result["동"].notnull()][result["폐지여부"] == "존재"]


,법정동코드,법정동명,폐지여부,시,구,동
2,1111010100,서울특별시 종로구 청운동,존재,서울특별시,종로구,청운동
3,1111010200,서울특별시 종로구 신교동,존재,서울특별시,종로구,신교동
4,1111010300,서울특별시 종로구 궁정동,존재,서울특별시,종로구,궁정동
5,1111010400,서울특별시 종로구 효자동,존재,서울특별시,종로구,효자동
6,1111010500,서울특별시 종로구 창성동,존재,서울특별시,종로구,창성동
...,...,...,...,...,...,...
1107,1174010600,서울특별시 강동구 둔촌동,존재,서울특별시,강동구,둔촌동
1108,1174010700,서울특별시 강동구 암사동,존재,서울특별시,강동구,암사동
1109,1174010800,서울특별시 강동구 성내동,존재,서울특별시,강동구,성내동
1110,1174010900,서울특별시 강동구 천호동,존재,서울특별시,강동구,천호동


In [444]:
seoul_dong[seoul_dong['동']=='신사동']

,법정동코드,법정동명,폐지여부,시,구,동
582,1138010900,서울특별시 은평구 신사동,존재,서울특별시,은평구,신사동
1043,1168010700,서울특별시 강남구 신사동,존재,서울특별시,강남구,신사동


In [446]:
seoul_dong = seoul_dong[['시', '구', '동']].reset_index(drop=True)
seoul_dong

,시,구,동
0,서울특별시,종로구,청운동
1,서울특별시,종로구,신교동
2,서울특별시,종로구,궁정동
3,서울특별시,종로구,효자동
4,서울특별시,종로구,창성동
...,...,...,...
462,서울특별시,강동구,둔촌동
463,서울특별시,강동구,암사동
464,서울특별시,강동구,성내동
465,서울특별시,강동구,천호동


In [452]:
ss_dong = seoul_dong['동'].unique()
for i in range(len(ss_dong)-1) :
    df1 = data[data['sidoname'] == ss_dong[0]].reset_index(drop=True)
    df2 = data[data['sidoname'] == ss_dong[i+1]].reset_index(drop=True)
    
    if i == 0 :
        seoul_data = pd.concat([df1, df2], axis=0)
    else :
        seoul_data = pd.concat([seoul_data, df2], axis=0)
        
seoul_data

,long,lat,order,hole,piece,id,group,sidoname
0,126.975560,37.589681,1,False,1,0,0.1,청운동
1,126.975491,37.589552,2,False,1,0,0.1,청운동
2,126.975456,37.589483,3,False,1,0,0.1,청운동
3,126.975444,37.589431,4,False,1,0,0.1,청운동
4,126.975424,37.589355,5,False,1,0,0.1,청운동
...,...,...,...,...,...,...,...,...
698,127.163865,37.575031,699,False,1,466,466.1,강일동
699,127.163763,37.575074,700,False,1,466,466.1,강일동
700,127.162922,37.575653,701,False,1,466,466.1,강일동
701,127.162556,37.575946,702,False,1,466,466.1,강일동


In [460]:
# 구, 행정동 컬럼 추가
seoul_data['구'] = 0
seoul_data['행정동'] = 0
seoul_data.head()

,long,lat,order,hole,piece,id,group,sidoname,구,행정동
0,126.975560,37.589681,1,False,1,0,0.1,청운동,0,0
1,126.975491,37.589552,2,False,1,0,0.1,청운동,0,0
2,126.975456,37.589483,3,False,1,0,0.1,청운동,0,0
3,126.975444,37.589431,4,False,1,0,0.1,청운동,0,0
4,126.975424,37.589355,5,False,1,0,0.1,청운동,0,0


# 카카오 API

In [462]:
from tqdm import tqdm

In [ ]:
url = 'https://dapi.kakao.com/v2/local/geo/coord2regioncode.json'
key = #'8a4a6d8eb9762860001214c19193a89d'
header = {'Authorization': 'KakaoAK {}'.format(key)}

# 경도, 위도 / X좌표, Y좌표, 상황에 맞춰 수정할 것
for x, y in tqdm(seoul_data[["long", "lat"]].values):
    queryString = {'x' : str(x), 'y' : str(y)}
    r = requests.get(url, headers=header, params=queryString)
    time.sleep(0.5)
    data = json.loads(r.text)
    
    hangjung = data["documents"][1]
    
    # 경도, 위도 / X좌표, Y좌표, 상황에 맞춰 수정할 것
    seoul_data.loc[(seoul_data["long"] == x) & (seoul_data["lat"] == y), "구"] = hangjung["region_2depth_name"]
    seoul_data.loc[(seoul_data["long"] == x) & (seoul_data["lat"] == y), "행정동"] = hangjung["region_3depth_name"]

  2%|█▌                                                                     | 5333/246000 [1:23:17<56:36:57,  1.18it/s]

In [ ]:
seoul_data

In [ ]:
for i in range(len(name)) :
    df1 = seoul_data[seoul_data['구'] == name[0]].reset_index(drop=True)
    df2 = seoul_data[seoul_data['구'] == name[i+1]].reset_index(drop=True)
    
    if i == 0 :
        seoul_df = pd.concat([df1, df2], axis=0)
    else :
        seoul_df = pd.concat([seoul_df, df2], axis=0)
        
seoul_df

In [ ]:
grid = pd.read_csv('data/KMeans/전체_그리드(2020, cp949).csv', encoding='cp949')
grid

In [ ]:
xy_df = pd.DataFrame()
xy_df['번째'] = grid.index
xy_df['X1'] = grid['X좌표(경도)'].str.split(' ~ ').str[0]
xy_df['X2'] = grid['X좌표(경도)'].str.split(' ~ ').str[1]
xy_df['Y1'] = grid['Y좌표(위도)'].str.split(' ~ ').str[0]
xy_df['Y2'] = grid['Y좌표(위도)'].str.split(' ~ ').str[1]
xy_df = xy_df.astype('float')
xy_df['번째'] = xy_df['번째'].astype('int64')
xy_df

In [ ]:
# 동 이름 리스트
dong_df = pd.read_csv('data/동별_면적_통계.csv', encoding='cp949')
dong_df = dong_df[(dong_df['동'] !='합계') & (dong_df['동'] != '소계')].reset_index(drop=True)
len(dong_df)

In [ ]:
dong_name = dong_df['동'].values
len(dong_name)

In [ ]:
def dong_xy_data_cnt(dong_name) :
    df = seoul_data[seoul_data['sidoname'] == dong_name]

    df = df.drop(['order', 'hole', 'piece', 'id', 'group'], axis=1)
    df.columns = ['X좌표', 'Y좌표', '행정동']

    df = df.drop_duplicates(['X좌표', 'Y좌표'], keep='first')

    cnt_list = []
    for i in df.index :
        X_coor = df['X좌표'][i]
        Y_coor = df['Y좌표'][i]

        result1 = xy_df[(xy_df["X1"] <= X_coor) & (xy_df["X2"] > X_coor)]
        result2 = result1[(result1["Y1"] <= Y_coor) & (result1["Y2"] > Y_coor)]

        if len(result2) == 0:
            cnt_list.append(0)
            continue

        cnt_list.append(result2["번째"].values[0])

    df['번째'] = cnt_list

    path = 'data/경계 좌표/동별/' + dong_name + ' 경계별 그리드 위치.csv'
    df.to_csv(path, encoding='cp949', index=False)

In [ ]:
for n in dong_name :
    xy_data_cnt(n)

In [ ]:
def count_grid(dong_name) :
    path = 'data/경계 좌표/' + dong_name + ' 경계별 그리드 위치.csv'
    
    data = pd.read_csv(path, encoding='cp949')
        
    data_cnt = data['번째'].unique()
    
    grid_cnt = pd.DataFrame()
    grid_cnt['번째'] = data_cnt
    
    x1_list = []
    x2_list = []
    y1_list = []
    y2_list = []

    for i in grid_cnt['번째'].values :
        x1_list.append(xy_df[xy_df.index == i]['X1'].values[0])
        x2_list.append(xy_df[xy_df.index == i]['X2'].values[0])
        y1_list.append(xy_df[xy_df.index == i]['Y1'].values[0])
        y2_list.append(xy_df[xy_df.index == i]['Y2'].values[0])

    grid_cnt['X1'] = x1_list
    grid_cnt['X2'] = x2_list
    grid_cnt['Y1'] = y1_list
    grid_cnt['Y2'] = y2_list
    
    data_cnt = grid_cnt['번째'].unique()

    grid_cnt = pd.DataFrame()
    grid_cnt['번째'] = data_cnt

    x1_list = []
    x2_list = []
    y1_list = []
    y2_list = []

    for i in grid_cnt['번째'].values :
        x1_list.append(xy_df[xy_df.index == i]['X1'].values[0])
        x2_list.append(xy_df[xy_df.index == i]['X2'].values[0])
        y1_list.append(xy_df[xy_df.index == i]['Y1'].values[0])
        y2_list.append(xy_df[xy_df.index == i]['Y2'].values[0])

    grid_cnt['X1'] = x1_list
    grid_cnt['X2'] = x2_list
    grid_cnt['Y1'] = y1_list
    grid_cnt['Y2'] = y2_list

    cnt = 0
    for i in range(len(grid_cnt)) :
        cnt += grid_cnt[(grid_cnt['X1']==grid_cnt['X1'][i])]['번째'].max() - grid_cnt[(grid_cnt['X1']==grid_cnt['X1'][i])]['번째'].min() + 1

    return cnt

In [ ]:
count_list = []
for n in dong_name :
    count_list.append(count_grid(n))
    
count_list

In [ ]:
cnt_df = pd.DataFrame()
cnt_df['행정동'] = dong_name
cnt_df['그리드_개수'] = count_list
cnt_df

In [ ]:
cnt_df.to_csv('data/경계 좌표/FIN/동별_그리드_개수.csv', index=False, encoding='cp949')